In [1]:
import pandas as pd

In [2]:
brfss = pd.read_csv("./temp/edit_brfss.csv")
checkup = pd.read_csv("./temp/건강검진.csv", encoding="cp949")
regionheart = pd.read_csv("./temp/지역별 심장질환.csv", encoding="cp949")
medical = pd.read_csv("./temp/hos_all.csv")

In [3]:
def check_dataframe(df):
    print(df.head())
    print(df.info())
    
    for col in df.columns:
        if len(df[col].unique()) <= 10:
            print(df[col].value_counts())
            print("*"*20)

## brfss

In [4]:
check_dataframe(brfss)

   HEARTDISEASE SEX  HEIGHT     WEIGHT  BLIND  DEAF  DIABETE  CHOLESTEROL  \
0           0.0   F  149.86  32.658651    0.0   0.0      0.0          1.0   
1           0.0   F  167.64        NaN    1.0   0.0      1.0          1.0   
2           0.0   F  165.10  77.110703    0.0   0.0      1.0          2.0   
3           0.0   F  162.56  88.450512    0.0   0.0      1.0          1.0   
4           1.0   M  180.34  93.440028    0.0   1.0      1.0          1.0   

   HYPERTENSION  KIDNEY  LIVER1  LIVER2  ALCOHOL  SMOKE  
0           0.0     0.0     NaN     NaN      0.0    1.0  
1           1.0     1.0     NaN     NaN      0.0    0.0  
2           1.0     0.0     NaN     NaN      0.0    0.0  
3           1.0     0.0     NaN     NaN      1.0    0.0  
4           0.0     0.0     NaN     NaN      0.0    0.0  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 438693 entries, 0 to 438692
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        -------------- 

### 열 제거
- CHOLESTEROL, LIVER 제거

In [5]:
brfss.drop(["CHOLESTEROL", "LIVER1", "LIVER2"], axis=1, inplace=True)

### 결측치 처리
- SMOKE 제외한 열 중에서 하나라도 결측치가 있는 행 제거

In [6]:
brfss.isnull().sum()

HEARTDISEASE     2655
SEX                 0
HEIGHT          24559
WEIGHT          40575
BLIND           17351
DEAF            16398
DIABETE           985
HYPERTENSION     1912
KIDNEY           1813
ALCOHOL         30589
SMOKE           24461
dtype: int64

In [7]:
brfss.isnull().sum() / len(brfss)

HEARTDISEASE    0.006052
SEX             0.000000
HEIGHT          0.055982
WEIGHT          0.092491
BLIND           0.039552
DEAF            0.037379
DIABETE         0.002245
HYPERTENSION    0.004358
KIDNEY          0.004133
ALCOHOL         0.069728
SMOKE           0.055759
dtype: float64

In [8]:
col_list = list(brfss.columns)
col_list.remove("SMOKE")

brfss.dropna(how="any", subset=col_list, inplace=True)

In [9]:
brfss["SMOKE"].fillna(9, inplace=True)

### HEIGHT, WEIGHT
- 검진자의 키(5CM 단위)
    - 예) 100-104CM --> 100CM
- 검진자의 몸무게(5KG 단위)
    - 예) 25~29KG -> 25KG

In [10]:
brfss["HEIGHT"] = brfss["HEIGHT"] // 5 * 5

In [11]:
brfss["WEIGHT"] = brfss["WEIGHT"] // 5 * 5

### 확인 및 저장

In [12]:
check_dataframe(brfss)

   HEARTDISEASE SEX  HEIGHT  WEIGHT  BLIND  DEAF  DIABETE  HYPERTENSION  \
0           0.0   F   145.0    30.0    0.0   0.0      0.0           0.0   
2           0.0   F   165.0    75.0    0.0   0.0      1.0           1.0   
3           0.0   F   160.0    85.0    0.0   0.0      1.0           1.0   
4           1.0   M   180.0    90.0    0.0   1.0      1.0           0.0   
5           0.0   M   190.0    85.0    0.0   0.0      0.0           0.0   

   KIDNEY  ALCOHOL  SMOKE  
0     0.0      0.0    1.0  
2     0.0      0.0    0.0  
3     0.0      1.0    0.0  
4     0.0      0.0    0.0  
5     0.0      0.0    1.0  
<class 'pandas.core.frame.DataFrame'>
Int64Index: 373951 entries, 0 to 438692
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   HEARTDISEASE  373951 non-null  float64
 1   SEX           373951 non-null  object 
 2   HEIGHT        373951 non-null  float64
 3   WEIGHT        373951 non-null  float64
 4   B

In [13]:
brfss.to_csv("../data/brfss.csv", index=False)

## 건강검진

In [14]:
check_dataframe(checkup)

   지역  SEX  HEIGHT  WEIGHT  BLIND  DEAF  DIABETE  HYPERTENSION  KIDNEY  LIVER  \
0  세종    1     165      60      0     0        0             0       0      0   
1  대구    2     150      65      0     0        0             0       1      0   
2  서울    2     155      55      0     0        0             0       1      0   
3  울산    1     160      70      0     1        1             0       0      1   
4  경기    2     155      50      0     0        0             1       1      0   

   ALCOHOL  SMOKE  
0      0.0      0  
1      0.0      0  
2      0.0      0  
3      0.0      0  
4      0.0      0  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2039244 entries, 0 to 2039243
Data columns (total 12 columns):
 #   Column        Dtype  
---  ------        -----  
 0   지역            object 
 1   SEX           int64  
 2   HEIGHT        int64  
 3   WEIGHT        int64  
 4   BLIND         int64  
 5   DEAF          int64  
 6   DIABETE       int64  
 7   HYPERTENSION  int64  
 8   KIDNE

In [15]:
# LIVER 제거
checkup.drop(["LIVER"], axis=1, inplace=True)

In [16]:
checkup["SEX"].replace({1:"M", 2:"F"}, inplace=True)

In [17]:
checkup.isnull().sum()

지역              0
SEX             0
HEIGHT          0
WEIGHT          0
BLIND           0
DEAF            0
DIABETE         0
HYPERTENSION    0
KIDNEY          0
ALCOHOL         0
SMOKE           0
dtype: int64

In [18]:
check_dataframe(checkup)

   지역 SEX  HEIGHT  WEIGHT  BLIND  DEAF  DIABETE  HYPERTENSION  KIDNEY  \
0  세종   M     165      60      0     0        0             0       0   
1  대구   F     150      65      0     0        0             0       1   
2  서울   F     155      55      0     0        0             0       1   
3  울산   M     160      70      0     1        1             0       0   
4  경기   F     155      50      0     0        0             1       1   

   ALCOHOL  SMOKE  
0      0.0      0  
1      0.0      0  
2      0.0      0  
3      0.0      0  
4      0.0      0  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2039244 entries, 0 to 2039243
Data columns (total 11 columns):
 #   Column        Dtype  
---  ------        -----  
 0   지역            object 
 1   SEX           object 
 2   HEIGHT        int64  
 3   WEIGHT        int64  
 4   BLIND         int64  
 5   DEAF          int64  
 6   DIABETE       int64  
 7   HYPERTENSION  int64  
 8   KIDNEY        int64  
 9   ALCOHOL       float64
 10 

In [19]:
checkup.to_csv("../data/checkup.csv", index=False, encoding="utf-8")

## 지역별 심장환자 현황

In [20]:
check_dataframe(regionheart)

      지역      사망자  심장장애
0    강원도   4858.0   130
1    경기도  25068.0  1312
2   경상남도   9972.0   262
3   경상북도  10292.0   306
4  광주광역시   2742.0   142
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   지역      17 non-null     object 
 1   사망자     17 non-null     float64
 2   심장장애    17 non-null     int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 536.0+ bytes
None


In [21]:
regionheart.isnull().sum()

지역      0
사망자     0
심장장애    0
dtype: int64

In [22]:
regionheart.to_csv("../data/region_heart.csv", index=False, encoding="utf-8")

## 의료수준

In [23]:
check_dataframe(medical)

   Unnamed: 0  지역  상급종합병원  종합병원   병원  심장혈관흉부외과  심장혈관흉부외과.1    일반의     전문의  \
0           0  서울      14    42  222      57.0         305  1,287  25,109   
1           1  부산       3    25  136      10.0          83    377   6,966   
2           2  인천       3    17   56       8.0          55    260   4,433   
3           3  대구       5    13   88       1.0          51    184   5,060   
4           4  광주       2    21   84       3.0          33    148   3,092   

      간호사  임상병리사  심장혈관흉부외과.2  심장혈관흉부외과.3   일반병실   일반병상  중환병실  중환병상  구급차 (대)  \
0  58,475  6,831        57.0         305  13846  45730   250  2660      177   
1  19,705  1,982        10.0          83   7646  24605    64   860       70   
2  12,551  1,343         8.0          55   3249  12248    47   610       55   
3  13,948  1,578         1.0          51   4580  16622    60   584       72   
4   9,414    941         3.0          33   4427  13894    27   408       30   

   구급대원수  
0   1473  
1    633  
2    495  
3    592  
4    27

In [24]:
# 첫 번째 열 삭제
medical = medical.iloc[:, 1:]

In [25]:
# 데이터 타입 변환
def change_to_int(x):
    return x.str.replace(",","").replace("-", "0").astype("int64")

medical[["일반의", "전문의", "간호사", "임상병리사"]] = medical[["일반의", "전문의", "간호사", "임상병리사"]].apply(change_to_int, axis=1)

In [26]:
medical.dtypes

지역             object
상급종합병원          int64
종합병원            int64
병원              int64
심장혈관흉부외과      float64
심장혈관흉부외과.1      int64
일반의             int64
전문의             int64
간호사             int64
임상병리사           int64
심장혈관흉부외과.2    float64
심장혈관흉부외과.3      int64
일반병실            int64
일반병상            int64
중환병실            int64
중환병상            int64
구급차 (대)         int64
구급대원수           int64
dtype: object

In [27]:
medical.to_csv("../data/medical.csv", index=False, encoding="utf-8")